In [1]:
!pip install emoji
!pip install tensorflow
!pip install transformers

     |████████████████████████████████| 175 kB 16.2 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=22ebee5628753bcfcfe77f56472dd8157b724c9426f7ae66ef627d886d8bc5da
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 462 kB 14.4 MB/s 
     |████████████████████████████████| 4.0 MB 14.2 MB/s 
     |████████████████████████████████| 895 kB 87.7 MB/s 
     |████████████████████████████████| 596 kB 90.6 MB/s 
     |████████████████████████████████| 77 kB 9.4 MB/s 
     |████████████████████████████████| 6.6 MB 79.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

model = TFAutoModelForSequenceClassification.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/finetuned-bertweet-sentiment-analysis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [4]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
link = 'https://drive.google.com/file/d/17-X8-Fg4B11bqxt_fZ5B-X2WzeuRky0f/view?usp=sharing' 

In [6]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Train_Feb.csv') 
 
df = pd.read_csv('Train_Feb.csv')
print(df)

                                                      text
0        @k_c12345 Brother this called link 👇\nhttps://...
1        Ukraine invasion: Who is President Volodymyr Z...
2        So it now looks like we're in a new phase of t...
3        @vanderbellen @karlnehammer TOP RUSSIAN AGGRES...
4        @ZelenskyyUa @EmmanuelMacron Ukraine won't fal...
...                                                    ...
1782546  @UNESCO Dear UNESCO. I'm a peaceful citizen of...
1782547  This is not Ukraine vs Russia war. This is a c...
1782548  Russia 🇷🇺 is warn... Don't provoke the 4nation...
1782549  Africa Report: How Africa factors in #RussiaUk...
1782550  @BanteSaket @LukeMackay13 @Reporter_indian I l...

[1782551 rows x 1 columns]


In [7]:
df['text'] = df['text'].astype('str') 

In [8]:
#df_sample = df.sample(n=50000, random_state=2)

In [9]:
#df_sample

In [10]:
docs = list(df.loc[:, "text"].values)

In [11]:
#get index for halving dataset at index: 178255
from math import floor
floor(len(docs) / 10)


178255

In [12]:
178255 * 8

1426040

In [13]:
docs = docs[1247785:1426040]

In [16]:
from tqdm import tqdm
scores = []

for i in tqdm(range(len(docs))):
    input = tokenizer.encode(docs[i], return_tensors="tf")
    output = model.predict(input)[0]
    prediction = tf.nn.softmax(output, axis=1).numpy()
    sentiment = np.argmax(prediction)
    
    scores.append({"Label": sentiment,
                       "Negative": prediction[0,0],
                       "Neutral": prediction[0,1],
                       "Positive": prediction[0,2]
                  })

100%|██████████| 178255/178255 [2:57:23<00:00, 16.75it/s]


In [17]:
df = df[1247785:1426040]

In [18]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score.set_index(df.index))

In [19]:
df

,text,Label,Negative,Neutral,Positive
1247785,"@AlexCobb_ Nah, all restrictions ended just be...",1,0.135641,0.854763,0.009596
1247786,#Zalensky BLATANTLY LIED on Twitter about #Tur...,0,0.941450,0.055986,0.002564
1247787,@meijer I'm calling on you to remove all Russi...,1,0.042827,0.749752,0.207421
1247788,@BorisJohnson STOP RUSSIAN AGGRESSION AGAINST ...,0,0.861700,0.127094,0.011206
1247789,@ua_parliament @NATO @eucopresident @vonderley...,0,0.974860,0.020367,0.004773
...,...,...,...,...,...
1426035,@cricket_ind1 @KremlinRussia_E So Keiling Sevi...,0,0.928839,0.068317,0.002844
1426036,Ukraine: The Most Dangerous Problem in the Wor...,1,0.172345,0.784978,0.042677
1426037,@markminervini @smigel_david Russia is not sca...,1,0.349432,0.631636,0.018932
1426038,@PatPromoterNFT Hey check out my NFT that I d...,1,0.013699,0.945666,0.040635


In [20]:
from google.colab import drive
drive.mount('drive')

df.to_csv('finetuned_Bertweet_SA_Feb_Train_p8.csv')
!cp finetuned_Bertweet_SA_Feb_Train_p8.csv "drive/My Drive/Colab Notebooks"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
